In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, ec06fb51-45fa-402a-9c37-11431406cec5, 4, Finished, Available, Finished)

In [3]:
import os
import shutil

#verifier la version du fichier à partir de son nom

bronzePath = "Files/Bronze/finess/finess.csv"
archiveFolder = "/lakehouse/default/Files/Bronze/archive"
bronzeFolder = "/lakehouse/default/Files/Bronze/finess"

#lire le fichier
bronze_df = spark.read.text(bronzePath)


#lire la date sur la premiere ligne du fichier
file_date = bronze_df.take(1)[0].value.split(';')[-1]
print("La date de MAJ du fichier est ", file_date)

#verifier si c'est la derniere version reçu
newFileName = "FINESS_DU_" + file_date + ".csv"

# Vérifier si le fichier existe dans le répertoire d'archive
if os.path.exists(os.path.join(archiveFolder, newFileName)):
    print("Le fichier existe déjà dans le répertoire d'archive.")
else:
    print("Le fichier n'existe pas dans le répertoire d'archive, il sera copié.")

    # Chemin du fichier source et destination
    sourceFilePath = os.path.join(bronzeFolder, "finess.csv")
    destinationFilePath = os.path.join(archiveFolder, newFileName)

    # Copier le fichier
    shutil.copy2(sourceFilePath, destinationFilePath)
    print(f"Fichier copié dans l'archive sous le nom : {newFileName}")

StatementMeta(, ec06fb51-45fa-402a-9c37-11431406cec5, 5, Finished, Available, Finished)

La date de MAJ du fichier est  2025-01-08
Le fichier existe déjà dans le répertoire d'archive.


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, split

# Initialisation de SparkSession
spark = SparkSession.builder \
    .appName("Lakehouse Data Processing with Split") \
    .getOrCreate()

# Chemin vers le fichier texte dans le Lakehouse
file_path = "Files/Bronze/finess/finess.csv"
colAll = split(col("value"), ";") 

# raw_data = spark.read.format("csv").option("header", "false").load(file_path)
raw_data = spark.read.text(file_path)
# display(raw_data)
# raw_data.show()
split_data = raw_data.withColumn("type", split(col("value"), ";").getItem(0)) 
                 
display(split_data)
structure_df = split_data.filter(col("type") == "structureet")
structureFinal_df = structure_df.withColumn("nofinesset",  colAll.getItem(1)) \
                                .withColumn("nofinessej" , colAll.getItem(2)) \
                                .withColumn("raison_social" , colAll.getItem(3)) \
                                .withColumn("raison_social_long" , colAll.getItem(4)) \
                                .withColumn("complrs" , colAll.getItem(5)) \
                                .withColumn("compldistrib" , colAll.getItem(6)) \
                                .withColumn("numvoie" , colAll.getItem(7)) \
                                .withColumn("typvoie" , colAll.getItem(8)) \
                                .withColumn("voie" , colAll.getItem(9)) \
                                .withColumn("compvoie" , colAll.getItem(10)) \
                                .withColumn("lieuditbp" , colAll.getItem(11)) \
                                .withColumn("commune" , colAll.getItem(12)) \
                                .withColumn("departement" , colAll.getItem(13)) \
                                .withColumn("ibdepartement" , colAll.getItem(14)) \
                                .withColumn("ligneacheminement" , colAll.getItem(15)) \
                                .withColumn("telephone" , colAll.getItem(16)) \
                                .withColumn("telecopie" , colAll.getItem(17)) \
                                .withColumn("categetab" , colAll.getItem(18)) \
                                .withColumn("libcategetab" , colAll.getItem(19)) \
                                .withColumn("categagretab" , colAll.getItem(20)) \
                                .withColumn("libcategagretab" , colAll.getItem(21)) \
                                .withColumn("siret" , colAll.getItem(22)) \
                                .withColumn("codeape" , colAll.getItem(23)) \
                                .withColumn("codemft" , colAll.getItem(24)) \
                                .withColumn("libmft" , colAll.getItem(25)) \
                                .withColumn("codesph" , colAll.getItem(26)) \
                                .withColumn("libsph" , colAll.getItem(27)) \
                                .withColumn("dateouv" , colAll.getItem(28)) \
                                .withColumn("dateautor" , colAll.getItem(29)) \
                                .withColumn("datemaj" , colAll.getItem(30)) \
                                .withColumn("numuai" , colAll.getItem(31)) \

# Filtrer les lignes contenant "geolocalisation"
geolocalisation_df = split_data.filter(col("type") == "geolocalisation")
# display(geolocalisation_df)
geolocalisationFinal_df = geolocalisation_df.withColumn("nofinesset",  colAll.getItem(1)) \
                                .withColumn("coordxet" , colAll.getItem(2)) \
                                .withColumn("coordyet" , colAll.getItem(3)) \
                                .withColumn("code" , split(colAll.getItem(4), ",").getItem(1)) \
                                .withColumn("sourcecoordet" , split(colAll.getItem(4), ",").getItem(2)) \
                                .withColumn("qualité" , split(colAll.getItem(4), ",").getItem(3)) \
                                .withColumn("base" , split(colAll.getItem(4), ",").getItem(4)) \
                                .withColumn("projection" , split(colAll.getItem(4), ",").getItem(4)) \
                                .withColumn("date" , colAll.getItem(5)) \
                                
# Afficher les résultats
print("DataFrame Structureet:")
df_structure = structureFinal_df.drop('value').drop('type')
display(df_structure)
df_geolocalisation = geolocalisationFinal_df.drop('value').drop('type')
print("DataFrame Geolocalisation:")
# geolocalisationFinal_df.show(truncate=False)
# display(geolocalisationFinal_df)
display(df_geolocalisation)



StatementMeta(, ec06fb51-45fa-402a-9c37-11431406cec5, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ea9ff77f-16c1-4ab9-8fc5-f44223d55f55)

DataFrame Structureet:


SynapseWidget(Synapse.DataFrame, bec242b1-989d-4f6d-b4b0-0cd964980bfd)

DataFrame Geolocalisation:


SynapseWidget(Synapse.DataFrame, 2cbb8b16-b690-4274-b913-e3000de308f2)

In [6]:
# Save data in dataframe
df_structure.write.format("delta").mode("overwrite").saveAsTable("finess_etablissement")
df_geolocalisation.write.format("delta").mode("overwrite").saveAsTable("finess_geolocalisation")

print(f"Les données sont nettoyé et enregistrer en delta table")

spark.sql("SHOW TABLES").show()

spark.stop()

StatementMeta(, ec06fb51-45fa-402a-9c37-11431406cec5, 8, Finished, Available, Finished)

Les données sont nettoyé et enregistrer en delta table
+---------------+--------------------+-----------+
|      namespace|           tableName|isTemporary|
+---------------+--------------------+-----------+
|sante_lakehouse|finess_etablissement|      false|
|sante_lakehouse|finess_geolocalis...|      false|
|sante_lakehouse|    person_activitie|      false|
+---------------+--------------------+-----------+

